### 국내 전종목 주가 데이터 크롤링

- 수정주가 크롤링

In [52]:
import keyring
import requests as rq
from io import BytesIO
import zipfile
import xmltodict
import json
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
ip = '43.201.116.164'

In [53]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(f'mysql+pymysql://jun:12345678@{ip}:3306/quant')
con = pymysql.connect(user='jun',
                      passwd='12345678',
                      host=f'{ip}',
                      db='quant',
                      charset='utf8')
mycursor = con.cursor()
# query = """
# select * from ticker
# where 기준일 = (select max(기준일) from ticker) and 종목구분 = '보통주';
# """
# ticker_list = pd.read_sql(query, con=engine)
# engine.dispose()
# ticker_list

In [54]:
ticker_list[ticker_list['종목코드'] == ticker]['종목명'].values[0]


'힘스'

In [40]:
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date

i = 0
ticker = ticker_list['종목코드'][i]
fr = (date.today() + relativedelta(years=-5)).strftime('%Y%m%d')
to = (date.today()).strftime('%Y%m%d')


# 네이버 수정주가 일단위 차트.

url = f'''https://api.finance.naver.com/siseJson.naver?symbol={ticker}&requestType=1&startTime={fr}&endTime={to}&timeframe=day'''
data = rq.get(url).content
data_price = pd.read_csv(BytesIO(data))
data_price # 주가 데이터


# 클랜징 처리
import re
price = data_price.iloc[:, 0:6]
price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
price = price.dropna()
price['날짜'] = price['날짜'].str.extract('(\d+)')
price['날짜'] = pd.to_datetime(price['날짜'])
price['종목코드'] = ticker
price['종목명'] = ticker_list[ticker_list['종목코드'] == ticker]['종목명'].values[0]
price

,날짜,시가,고가,저가,종가,거래량,종목코드,종목명
0,2018-09-20,6810.0,7120.0,6670.0,7030.0,181397.0,095570,AJ네트웍스
1,2018-09-21,7110.0,7130.0,6270.0,6490.0,359049.0,095570,AJ네트웍스
2,2018-09-27,6600.0,6600.0,6220.0,6360.0,210740.0,095570,AJ네트웍스
3,2018-09-28,6530.0,6530.0,5980.0,6160.0,171325.0,095570,AJ네트웍스
4,2018-10-01,6160.0,6350.0,6050.0,6180.0,77744.0,095570,AJ네트웍스
...,...,...,...,...,...,...,...,...
1229,2023-09-14,4515.0,4650.0,4425.0,4535.0,629381.0,095570,AJ네트웍스
1230,2023-09-15,4645.0,4680.0,4480.0,4625.0,586258.0,095570,AJ네트웍스
1231,2023-09-18,4630.0,4630.0,4260.0,4320.0,723083.0,095570,AJ네트웍스
1232,2023-09-19,4320.0,4500.0,4285.0,4460.0,325245.0,095570,AJ네트웍스


In [42]:
len(ticker_list)

2372

In [61]:

# 티커 리스트 불러오기
ticker_list = pd.read_sql("""
                    select * from ticker
                    where 기준일 = (select max(기준일) from ticker) and 종목구분 = '보통주';
                    """, con=engine)                          

# 오류 발생시 저장할 리스트 생성
error_list = []

# 전종목 주가 다운로드 및 저장
for i in tqdm(range(0, len(ticker_list))):
    
    # 티커 선택
    ticker = ticker_list['종목코드'][i]
    
    # 시작일과 종료일
    # fr = (date.today() + relativedelta(years=-5)).strftime('%Y%m%d')
    fr = (date.today() + relativedelta(days = -1)).strftime("%Y%m%d")
    to = (date.today()).strftime('%Y%m%d')
    
    
    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:
    
        # url 생성
        url = f'https://api.finance.naver.com/siseJson.naver?symbol={ticker}&requestType=1&startTime={fr}&endTime={to}&timeframe=day'
        
        # 데이터 다운로드
        data = rq.get(url).content
        data_price = pd.read_csv(BytesIO(data))
        
        # 데이터 클랜징
        price = data_price.iloc[:, 0:6]
        price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
        price = price.dropna()
        price['날짜'] = price['날짜'].str.extract('(\d+)')
        price['날짜'] = pd.to_datetime(price['날짜'])
        price['종목코드'] = ticker
        price['종목명'] = ticker_list[ticker_list['종목코드'] == ticker]['종목명'].values[0]

        # 주가 데이터를 DB에 저장
        # price.to_sql(name="stock_price", con=engine, index=True, if_exists='append')
        
    except:
        
        # 오류 발생 시 error_list에 티커 저장하고 넘어가기
        print(ticker)
        error_list.append(ticker)
    
    # 타임슬립 적용
    time.sleep(1)


  1%|          | 17/2372 [00:22<50:51,  1.30s/it]


KeyboardInterrupt: 

In [50]:
stock_code = pd.read_sql("""
select * from stock_price;
""", con = engine)

In [45]:
len(stock_code)

2671745

In [63]:
stock_code.to_sql(name="stock_price", con=engine, index=True, if_exists='replace')

#2671745 rows × 9 columns

2671745

In [62]:
price

<bound method NDFrame.to_sql of           날짜      시가      고가      저가      종가       거래량    종목코드      종목명
0 2023-09-20  8200.0  8240.0  8090.0  8160.0  399018.0  139130  DGB금융지주
1 2023-09-21  8080.0  8160.0  8010.0  8060.0  263521.0  139130  DGB금융지주>